<a href="https://colab.research.google.com/github/ml2-picme/PicMe/blob/master/Find_Images_In_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This Notebook loads the content of the shared Google Drive directory into the lokal session storage

### 1. Load the content of the Drive directory into local storage

In [4]:
# Install the PyDrive wrapper & import libraries
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Delete the local directory
!rm -r /root/data/
print("Deleted local directory")

# Set a local directory and upload files from Google Drive
local_root_path = os.path.expanduser("~/data")
try:
  os.makedirs(local_root_path)
except: pass

def ListFolder(google_drive_id, destination):
  file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % google_drive_id}).GetList()
  counter = 0
  for f in file_list:
    # If it is a directory then, create the dicrectory and upload the file inside it
    if f['mimeType']=='application/vnd.google-apps.folder': 
      folder_path = os.path.join(destination, f['title'])
      os.makedirs(folder_path)
      print('* creating directory {}'.format(folder_path))
      ListFolder(f['id'], folder_path)
    else:
      fname = os.path.join(destination, f['title'])
      f_ = drive.CreateFile({'id': f['id']})
      f_.GetContentFile(fname)
      counter += 1
  print('* {} files were uploaded in {}'.format(counter, destination))

# Opening and uploading the Drive Folder => this is the shared directory and should work for everyone who belongs to the project
print("\nCalling the ListFolder method, that uploads the data...")
ListFolder("1PTOfzcdMkVn5bTJ6nhGALQO8ICGBGbnc", local_root_path)
print("Upload finished!")

# Show the Data in the local Path
print("\nListing files in local test directory")
!ls -l /root/data/test_images

Deleted local directory

Calling the ListFolder method, that uploads the data...
* creating directory /root/data/test_images
* 8 files were uploaded in /root/data/test_images
* 0 files were uploaded in /root/data
Upload finished!

Listing files in local test directory
total 1076
-rw-r--r-- 1 root root 106647 Mar 26 15:11 auto.jpg
-rw-r--r-- 1 root root 204846 Mar 26 15:11 berge.jpg
-rw-r--r-- 1 root root 104237 Mar 26 15:11 brücke.jpg
-rw-r--r-- 1 root root 118569 Mar 26 15:11 handy.jpg
-rw-r--r-- 1 root root  51903 Mar 26 15:11 hund.jpg
-rw-r--r-- 1 root root  53904 Mar 26 15:11 papagei.jpg
-rw-r--r-- 1 root root  97310 Mar 26 15:11 sandwich.jpg
-rw-r--r-- 1 root root 346140 Mar 26 15:11 waffe.jpg


### 2. Make a list of paths for the image files in this directory (and it's subdirectories)

In [5]:
import os

path = '/root/data'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if file.endswith(".jpg"):
            files.append(os.path.join(r, file))
            
print("Printing files:")

for f in files:
    print(f)

Printing files:
/root/data/test_images/auto.jpg
/root/data/test_images/brücke.jpg
/root/data/test_images/papagei.jpg
/root/data/test_images/handy.jpg
/root/data/test_images/hund.jpg
/root/data/test_images/berge.jpg
/root/data/test_images/waffe.jpg
/root/data/test_images/sandwich.jpg


### 3. Classify the Images using pre-trained model

In [7]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16

from PIL import Image
import urllib.request

# load the model
model = VGG16()

for f in files:
    # load an image from file
    image = load_img(f, target_size=(224, 224))
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    # predict the probability across all output classes
    yhat = model.predict(image)
    # convert the probabilities to class labels
    label = decode_predictions(yhat)
    # retrieve the most likely result, e.g. highest probability
    label = label[0][0]
    # print the classification
    print("Following classes were predicted:")
    
    load_img(f)
    
    
    
    print('%s --> %s (%.2f%%)' % (f, label[1], label[2]*100))

40960/35363 [==================================] - 0s 0us/step
/root/data/test_images/auto.jpg --> sports_car (59.34%)
/root/data/test_images/brücke.jpg --> steel_arch_bridge (96.87%)
/root/data/test_images/papagei.jpg --> macaw (99.62%)
/root/data/test_images/handy.jpg --> iPod (53.37%)
/root/data/test_images/hund.jpg --> Labrador_retriever (49.45%)
/root/data/test_images/berge.jpg --> alp (66.41%)
/root/data/test_images/waffe.jpg --> revolver (81.78%)
/root/data/test_images/sandwich.jpg --> French_loaf (41.49%)
